# This code implements the training pipeline for various models

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from dataformatter import *
from models import *
import matplotlib.pyplot as plt
import os
import pdb
import random
from collections import defaultdict

In [2]:
def set_random_seed(seed):
	# Esp important for ensuring deterministic behavior with CNNs
	torch.backends.cudnn.deterministic = True
	np.random.seed(seed)
	random.seed(seed)
	torch.manual_seed(seed)
	cuda_available = torch.cuda.is_available()
	if cuda_available:
		torch.cuda.manual_seed_all(seed)
	return cuda_available

In [3]:
def run_epoch(model, optimizer, data_iterator, mode='train', eval_frac=-1):
    model.train()
    if mode == 'val':
        model.eval()
    stats = []
    max_tr_batch = -1
    num_egs, batch_idx, n_ones = 0, 1, 0
    for batch in data_iterator:
        # we get the loss from passing the batch to the model
        # each model will have it's own way of deadling with the data [we can jointly figure this out]
        if eval_frac > 0:
            batch, num_batches = batch
            max_tr_batch = int(eval_frac * num_batches)
            n_ones = (np.array(batch)[:, -1]).sum()
        loss, acc, bsz = model(batch)
        stats.append([loss.item(), acc.item(), bsz, n_ones, len(batch)])
        if mode == 'train' and ((batch_idx < max_tr_batch) or (max_tr_batch < 0)):
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
        batch_idx += 1
    stats = np.array(stats)
    avg_loss = np.mean(stats[:, 0])
    avg_acc = (stats[:, 1]).sum() / (stats[:, 2].sum() * 1.0)
    return (avg_loss, avg_acc), stats[max_tr_batch:, :]

In [4]:
def trainer(model, optimizer, dataset, num_epochs=20, desc='Description of model', eval_frac=0.8, shuffle=True):
    # Todo [all]
    # Figure out how to split the data into a train-val-test regime
    stats = []
    for epoch_ in range(num_epochs):
        # get a data iterator for this epoch
        data_iter = get_batch_iterator(dataset, BATCH_SZ, shuffle=shuffle, batch_info=True)
        epoch_stats, e_stats = run_epoch(model, optimizer, data_iter, mode='train', eval_frac=eval_frac)
        stats.append(epoch_stats)
        print('Epoch {} : Avrg Loss = {}, Avrg Acc = {} '.format(epoch_, stats[-1][0], stats[-1][1]))
        major_acc = (e_stats[:, 3].sum()) / (1.0 * e_stats[:, 4].sum())
        major_acc = max(major_acc, 1.0 - major_acc)
        print('Epoch {} : Eval Loss = {}, Eval Acc = {}, Eval Majority Acc = {}'.format(epoch_, (e_stats[:, 0]).mean(), (e_stats[:, 1].sum())/(1.0 * e_stats[:, 2].sum()), major_acc))
        print('-'*50)

    stats = np.array(stats)
#     graph_results(stats, desc)
    return model

def set_wise_trainer(model, optimizer, setwise_dataset, num_epochs=20, desc='Set-Wise Model', shuffle=False):
    model.train()
    stats = defaultdict(list)
    for epoch_ in range(num_epochs):
        # get a data iterator for this epoch
        accs = []
        setwise_keys = list(setwise_dataset.keys())
        perm = np.random.permutation(len(setwise_keys))
        setwise_keys = np.array(setwise_keys)[perm]
        for set_id  in setwise_keys:
            dataset = setwise_dataset[set_id]
            model.remap_embedders(dataset, set_id)
            data_iter = get_batch_iterator(dataset, BATCH_SZ, shuffle=shuffle)
            this_stats, _ = run_epoch(model, optimizer, data_iter, mode='train')
            stats[set_id].append(this_stats)
            accs.append(this_stats[-1])
        acc_stats = np.min(accs), np.mean(accs), np.median(accs), np.max(accs)
        print('Min Acc {} | Mean Acc : {} | Median Acc {} | Max Acc {} '.format(*acc_stats))
    return model, stats

def evaluate(model, dataset, epoch_=-1, print_res=True, shuffle=False):
    # get a data iterator for this epoch
    model.eval()
    data_iter = get_batch_iterator(dataset, BATCH_SZ, shuffle=shuffle, batch_info=False)
    epoch_stats, _ = run_epoch(model, None, data_iter, mode='test')
    if print_res:
        print('Epoch {} : Avg Loss = {} Avg Acc = {}'.format(epoch_, epoch_stats[0], epoch_stats[1]))
    return epoch_stats

In [5]:
def eval_setwise(eval_setwise_dataset, model, MAX_TR_KEY=0, desc='graph', epoch=-1):
    all_stats = []
    for id_, (set_id, this_dataset) in enumerate(eval_setwise_dataset.items()):
        set_id = MAX_TR_KEY + int(set_id)
        model.remap_embedders(this_dataset, set_id)
        result = evaluate(model, this_dataset, print_res=False)
        average_pred = np.mean(this_dataset[:, -1])
        all_stats.append([*result, 1.0 - average_pred, average_pred])
    av_res = np.mean(all_stats, axis=0)
    print('[{}] Epoch[{}] : Loss {}, Acc {}, Major [0] Acc {}, Marjor [1] Acc {}'.format(desc, epoch, *av_res))

In [6]:
DATA_PATH = 'hawkeye_trace_belady_graph.csv' # This is the CSV FILE WE ARE TRYING TO ANALYZE
TR_DESC = 'GRAPH'
EVAL_DATA_PATH = 'hawkeye_trace_belady_bzip.csv' # This is the CSV FILE WE ARE TRYING TO ANALYZE
EVAL_DESC = 'BZIP'
SAVE_FLDR = 'results'
N_EPOCHS = 5
MAX_GRAD_NORM = 0.1
SET_WISE = True
RANDOM_SEED = 140982301

In [7]:
def model_main():
    if not os.path.exists(SAVE_FLDR):
        os.makedirs(SAVE_FLDR)

    set_random_seed(RANDOM_SEED)
    print('Creating Model of type : {}, Batchsz = {}, Learning Rate = {}'.format(MODEL_TYPE, BATCH_SZ, LR))
    model = get_model(MODEL_TYPE)
    chosen_columns = model.get_data_columns()
    train_dataset = csv_to_data(DATA_PATH, chosen_columns)
    average_pred = np.mean(train_dataset[:, -1])
    if torch.cuda.is_available():
        model.cuda()
    model.use_cuda = torch.cuda.is_available()
    eval_dataset = csv_to_data(EVAL_DATA_PATH, chosen_columns)
    print(model.feat_idx_map, torch.cuda.is_available())
    print('This is the average accuracy : ', 1.0 - average_pred, ' From predicting all zeros')
    
    if not SET_WISE:
        optimizer = Adam(model.parameters(), lr=LR)
        model.prep_for_data(train_dataset, temp_order=True)
        model = trainer(model, optimizer, train_dataset, num_epochs=N_EPOCHS, desc=MODEL_DESC, shuffle=False)
    else:
        train_setwise_dataset = group_by_set(train_dataset)
        eval_setwise_dataset = group_by_set(eval_dataset)
        all_tr_keys = list(train_setwise_dataset.keys())
        val_keys = np.random.choice(all_tr_keys, size=int(0.2 * len(all_tr_keys)))
        tr_keys = set(all_tr_keys) - set(val_keys)

        vals = [len(x) for x in list(train_setwise_dataset.values())]
        max_key = all_tr_keys[np.argmax(vals)]
        # Logging-pre-training performance
        model.prep_for_data(train_setwise_dataset[max_key], temp_order=True)
        for set_id, this_dataset in train_setwise_dataset.items():
            model.remap_embedders(this_dataset, set_id)

        tr_val_setwise_dataset = {k: train_setwise_dataset[k] for k in val_keys}
        train_setwise_dataset = {k: train_setwise_dataset[k] for k in tr_keys}

        print('Logging Pre-Training Performance')
        MAX_TR_KEY = max([int(x) for x in train_setwise_dataset.keys()]) + 1
        eval_setwise(eval_setwise_dataset, model, MAX_TR_KEY=MAX_TR_KEY, desc=EVAL_DESC)
        eval_setwise(tr_val_setwise_dataset, model, desc=TR_DESC)
        optimizer = Adam(model.parameters(), lr=LR) # Now we can add all the model parameters to the optimizer
        for i in range(N_EPOCHS):
            model, stats = set_wise_trainer(model, optimizer, train_setwise_dataset, num_epochs=1)
            eval_setwise(eval_setwise_dataset, model, MAX_TR_KEY=MAX_TR_KEY, desc=EVAL_DESC, epoch=i)
            eval_setwise(tr_val_setwise_dataset, model, desc=TR_DESC, epoch=i)
    torch.save(model.state_dict(), '{}/{}_saved_model.pth'.format(SAVE_FLDR, MODEL_DESC))
    # Neeed to return the eval performance here

In [8]:
batch_szs = [32, 64]
lrs = [1e-4, 3e-4]
model_types = ['TRANSFORMER', "TRANSFORMER_1", "TRANSFORMER_2"]

for BATCH_SZ in batch_szs:
    for LR in lrs:
        for MODEL_TYPE in model_types:
            MODEL_DESC = "{}-{}_BSZ.{}_LR.{}".format(TR_DESC, MODEL_TYPE, BATCH_SZ, LR)
            model_main()

Creating Model of type : TRANSFORMER, Batchsz = 32, Learning Rate = 0.0001
{'Program Counter': 0, 'Set Occupancy': 1, 'Belady Friendly': 2} True
This is the average accuracy :  0.6618220085895503  From predicting all zeros
Logging Pre-Training Performance
[BZIP] Epoch[-1] : Loss 0.704139049822125, Acc 0.5210902236125422, Major [0] Acc 0.656989378278163, Marjor [1] Acc 0.3430106217218373
[GRAPH] Epoch[-1] : Loss 0.7683464638166018, Acc 0.3600384794776119, Major [0] Acc 0.026021041058635064, Marjor [1] Acc 0.973978958941365
Min Acc 0.9272463768115942 | Mean Acc : 0.9586082872660178 | Median Acc 0.9562878787878788 | Max Acc 0.99640625 
[BZIP] Epoch[0] : Loss 2.4962187632012407, Acc 0.3850104996146359, Major [0] Acc 0.656989378278163, Marjor [1] Acc 0.3430106217218373
[GRAPH] Epoch[0] : Loss 0.13434152012067993, Acc 0.9710202891791044, Major [0] Acc 0.026021041058635064, Marjor [1] Acc 0.973978958941365
Min Acc 0.9272463768115942 | Mean Acc : 0.9599671433353215 | Median Acc 0.9582858435097

In [ ]:
chosen_columns = ['Program Counter', 'Physical Address', 'Set', 'Cache Friendly']
dataset = csv_to_data(DATA_PATH, chosen_columns)
pcs = dataset[:, 0]
dict_ = defaultdict(int)
for id_ in pcs:
    dict_[id_] += 1
values = np.array(list(dict_.values()))
bc = np.bincount(values)
print(len(values), dataset.shape, values.mean(), values.max(), values.min(), np.median(values))
print(bc)